<a href="https://colab.research.google.com/github/Saify-Technologies/MNIST-example-deep-learning-in-googles-jax/blob/main/full_connected_network_in_google_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
from IPython.core.debugger import set_trace

import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from fastai.vision import * 
from fastai.data.all import *
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from PIL import Image
from numpy import asarray


MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [ ]:
from PIL import Image
from numpy import asarray
def get_data(validation_per):
    path = untar_data(URLs.MNIST);
    #creating List
    x_train_ls=[]
    y_train_ls = []
    for i in range(0,10):
      x_ls ,y_ls = [asarray(Image.open(f)) for f in (path/"training"/str(i)).ls()] ,[ i for f in (path/"training"/str(i)).ls()]
      y_train_ls.extend(y_ls)
      x_train_ls.extend(x_ls)

    #converting list to numpy
    train_x ,train_y = np.array(x_train_ls) , np.array(y_train_ls)
    
    #randomly shuffling data in th numpy
    seed = np.random.randint(0, 10000)
    np.random.seed(seed)
    np.random.shuffle(train_x)
    np.random.seed(seed)
    np.random.shuffle(train_y)

    #dividing validation and training 
    training_set= int(train_x.shape[0] * (1-(validation_per/100)))
    training_x,training_y,validation_x,validation_y = train_x[:training_set],train_y[:training_set],train_x[training_set:],train_y[training_set:]

    return map(jnp.array, (training_x,training_y,validation_x,validation_y))


In [ ]:
x_train , y_train , x_valid,y_valid = get_data(20)

In [ ]:
def normalize(x, m, s): return (x-m)/s

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(DeviceArray(33.300873, dtype=float32), DeviceArray(78.545265, dtype=float32))

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
import numpy as np
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]) #flatten rows to make it 784 last two columns are merged
x_valid =  x_valid.reshape(x_valid.shape[0], x_valid.shape[1]*x_valid.shape[2])
x_train.shape, x_valid.shape

((48000, 784), (12000, 784))

In [ ]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(48000, 784, DeviceArray(10, dtype=int32))

In [ ]:
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [ ]:
#xavier initialization
nh = 50 #number of hidden units will be 50
seed=1
key = random.PRNGKey(seed)
w1 = random.normal(key=key,shape=(m,nh)) / math.sqrt(m)
b1 = jnp.zeros(nh)
w2 = random.normal(key=key,shape=(nh ,1)) / math.sqrt(nh)
b2 = jnp.ones(1)

In [ ]:
def lin(x,w,b):
  return x@w + b

In [ ]:
def relu(x):
    return jnp.maximum(0.,x)

In [ ]:
nh = 50 #number of hidden units will be 50
seed=1
key = random.PRNGKey(seed)
w1 = random.normal(key=key,shape=(m,nh)) * math.sqrt(2/m)
b1 = jnp.zeros(nh)
w2 = random.normal(key=key,shape=(nh ,1)) * math.sqrt(2/nh)
b2 = jnp.ones(1)

In [ ]:
def leakyrelu(x): return jnp.maximum(0.,x) - 0.5

In [ ]:
t = leakyrelu(lin(x_valid,w1,b1)) #this layer should have mean and variance 0 and 1 respectively which is there with leaky relu
t.mean() , t.std()

(DeviceArray(-0.03897044, dtype=float32),
 DeviceArray(0.7305525, dtype=float32))

In [ ]:
def model(xb,w1,w2,b1,b2) :
  l1=lin(xb,w1,b1)
  l2=leakyrelu(l1)
  l3=lin(l2,w2,b2)
  return l3

In [ ]:
def mse(x,y):
  return ((jnp.squeeze(x) - y)**2).mean()

In [ ]:
mse(model(x_valid,w1,w2,b1,b2),y_valid)

DeviceArray(20.31205, dtype=float32)

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')

class Relu(Module):
    def forward(self, inp): return jnp.maximum(0.,inp) - 0.5

class Lin(Module):
    def __init__(self, w, b): 
      self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b

class Mse(Module):
    def forward (self, inp, targ): return ((jnp.squeeze(inp) - targ)**2).mean()

In [ ]:
def Model(x,targ,w1,b1,w2,b2):
      layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
      loss = Mse()
      for l in layers: x = l(x)
      return loss(x, targ)
      
lr = 0.001      



In [ ]:
import jax
# model = Model(w1,b1,w2,b2)

In [ ]:
loss , grads = jax.value_and_grad(Model,argnums=[2,3,4,5])(x_valid,y_valid,w1,b1,w2,b2)

In [ ]:
loss ,grads

(DeviceArray(20.31205, dtype=float32),
 (DeviceArray([[-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
                -0.02454017,  0.16890837],
               [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
                -0.02454017,  0.16890837],
               [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
                -0.02454017,  0.16890837],
               ...,
               [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
                -0.02454017,  0.16890837],
               [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
                -0.02454017,  0.16890837],
               [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
                -0.02454017,  0.16890837]], dtype=float32),
  DeviceArray([ 0.17495301, -0.5272211 , -0.23879513,  0.2152882 ,
               -0.762972  , -0.4169765 , -1.6098483 , -2.0278134 ,
                0.15746224,  0.32992813, -0.68278205, -0.21884523,
                0.8280484 ,

In [ ]:
grads

(DeviceArray([[-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
               -0.02454017,  0.16890837],
              [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
               -0.02454017,  0.16890837],
              [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
               -0.02454017,  0.16890837],
              ...,
              [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
               -0.02454017,  0.16890837],
              [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
               -0.02454017,  0.16890837],
              [-0.07417491,  0.22352615,  0.10124209, ...,  0.04201994,
               -0.02454017,  0.16890837]], dtype=float32),
 DeviceArray([ 0.17495301, -0.5272211 , -0.23879513,  0.2152882 ,
              -0.762972  , -0.4169765 , -1.6098483 , -2.0278134 ,
               0.15746224,  0.32992813, -0.68278205, -0.21884523,
               0.8280484 ,  0.0088369 , -0.07505147, -1.3169811 ,
              -0

In [ ]:
def nll(input,target): return -input[[i for i in range(input.shape[0])],target].mean()

In [ ]:
class NegativeLogLikelyhood(Module):
    def forward (self, inp, targ): return -inp[[i for i in range(inp.shape[0])],targ].mean()

In [ ]:
def log_softmax(x):
    return jnp.log(jnp.exp(x) / jnp.sum(jnp.exp(x)))

In [ ]:
nh = 50 #number of hidden units will be 50
seed=1
key = random.PRNGKey(seed)
w1 = random.normal(key=key,shape=(m,nh)) * math.sqrt(2/m)
b1 = jnp.zeros(nh)
w2 = random.normal(key=key,shape=(nh ,10)) * math.sqrt(2/nh)
b2 = jnp.ones(10)

def Model(x,targ,w1,b1,w2,b2):
      layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
      loss = NegativeLogLikelyhood()
      for l in layers: x = l(x)
      return loss(log_softmax(x), targ)

In [ ]:
loss , grads = jax.value_and_grad(Model,argnums=[2,3,4,5])(x_valid,y_valid,w1,b1,w2,b2)

In [ ]:
lr=0.5
epochs=1
pred = model(x_train , w1,w2,b1,b2)
totalNumerOfRows= pred.shape[0]

In [ ]:
totalNumerOfRows

48000

In [ ]:
nh = 50 #number of hidden units will be 50
seed=1
bs=64
key = random.PRNGKey(seed)
w1 = random.normal(key=key,shape=(m,nh)) * math.sqrt(2/m)
b1 = jnp.zeros(nh)
w2 = random.normal(key=key,shape=(nh ,10)) * math.sqrt(2/nh)
b2 = jnp.ones(10)
lr=0.01
epochs = 5
loss , grads = jax.value_and_grad(Model,argnums=[2,3,4,5])(x_train[0:64],y_train[0:64],w1,b1,w2,b2)
print("Initial Loss:-"+loss)

for epoch in range(epochs):
  for i in range(((totalNumerOfRows-1)//bs)+1):
    start_index=i*bs
    end_index=(i+1)*bs
    loss , grads = jax.value_and_grad(Model,argnums=[2,3,4,5])(x_train[start_index:end_index],y_train[start_index:end_index],w1,b1,w2,b2)
    w1 -= lr*grads[0]
    b1 -= lr*grads[1]
    w2 -= lr*grads[2]
    b2 -= lr*grads[3]
    
  print(loss)
    # print("accuracy:"+str(accuracy_myself(pred,y_train[start_index:end_index]))) 

4.6549664
4.5501685
4.5036073
4.4762754
4.456649
